In [1]:
from scipy.stats import percentileofscore as pctrank
from datetime import datetime
import pandas as pd
import numpy as np
import sys, os

In [2]:
DATE = "2010-01-01"

### Data

In [3]:
products = pd.read_csv("data/vol_products.csv")
products = products.set_index("Ticker")["Link"].to_dict()
products

{'RVOL': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL_History.csv',
 'RVOL3M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL3M_History.csv',
 'RVOL6M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL6M_History.csv',
 'RVOL12M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL12M_History.csv',
 'VIX': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIXcurrent.csv',
 'VXD': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VXDohlcprices.csv',
 'VXN': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VXNcurrent.csv',
 'RVX': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVXdailyprices.csv',
 'VIX3M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIX3Mdailyprices.csv',
 'VIX6M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIX6Mdailyprices.csv'}

### Loaders

In [4]:
def rvol(key):
    rv = pd.read_csv(products[key])
    rv['Date'] = pd.to_datetime(rv.Date.values)
    rv = rv.sort_values('Date', ascending=True)
    rv.columns = ['date', 'val']
    return rv

def get_vixm(key):
    v = pd.read_csv(products[key], skiprows=2)
    v.columns = ['date', 'open', 'high', 'low', 'val']
    v['date'] = pd.to_datetime(v.date.values)
    return v[['date', 'val']]

def get_major(key, name, skip):
    v = pd.read_csv(products[key], skiprows=skip)
    v['Date'] = pd.to_datetime(v.Date.values)
    v = v[['Date', name]]
    v.columns = ['date', 'val']
    return v

### Downloads

In [5]:
rv, rv3m, rv6m = rvol("RVOL"), rvol("RVOL3M"), rvol("RVOL6M")

In [6]:
vix3m, vix6m = get_vixm("VIX3M"), get_vixm("VIX6M")

In [7]:
vix = get_major("VIX", "VIX Close", 1)
vxn = get_major("VXN", "Close", 2)
rvx = get_major("RVX", "Close", 2)
vxd = get_major("VXD", "Close", 4)

### Spot Indices

In [8]:
URL = "https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={p1}&period2={p2}&interval=1d&events=history&includeAdjustedClose=true"

dt = datetime.now()

p1 = datetime(dt.year-15, dt.month, dt.day)
p2 = datetime(dt.year, dt.month, dt.day)

p1 = int(p1.timestamp() / 1000) * 1000
p2 = int(p2.timestamp() / 1000) * 1000

In [29]:
def get_index(ticker, p1, p2):
    index = pd.read_csv(URL.format(ticker=ticker, p1=p1, p2=p2))[['Date', 'Adj Close']]
    index['Date'] = pd.to_datetime(index.Date.values)
    index.columns = ['date', 'val']
    return index[index.date >= DATE].reset_index(drop=True)

def calculate_rv(data, days):
    x = np.log(data.val / data.val.shift()) ** 2
    x = x.rolling(days, min_periods=1).sum()
    return np.sqrt(x * (252 / days)) * 100

In [28]:
spx = get_index("%5EGSPC", p1, p2)
rut = get_index("%5ERUT", p1, p2)
dji = get_index("%5EDJI", p1, p2)
ndx = get_index("%5ENDX", p1, p2)

In [30]:
spxrv = calculate_rv(spx, 21)
spxrv3m = calculate_rv(spx, 63)
spxrv6m = calculate_rv(spx, 126)

rutrv = calculate_rv(rut, 21)
ndxrv = calculate_rv(ndx, 21)
djirv = calculate_rv(dji, 21)

0             NaN
1        0.631965
2        0.641603
3        1.034280
4        1.188053
          ...    
2738    21.221162
2739    21.342691
2740    21.346280
2741    21.294580
2742    21.312691
Name: val, Length: 2743, dtype: float64


### Filter by Date

In [12]:
rv = rv[rv.date >= DATE].reset_index(drop=True)
rv3m = rv3m[rv3m.date >= DATE].reset_index(drop=True)
rv6m = rv6m[rv6m.date >= DATE].reset_index(drop=True)
vix = vix[vix.date >= DATE].reset_index(drop=True)
vix3m = vix3m[vix3m.date >= DATE].reset_index(drop=True)
vix6m = vix6m[vix6m.date >= DATE].reset_index(drop=True)
vxn = vxn[vxn.date >= DATE].reset_index(drop=True)
rvx = rvx[rvx.date >= DATE].reset_index(drop=True)
vxd = vxd[vxd.date >= DATE].reset_index(drop=True)

### RVX - SPX

In [23]:
def pair_stats(ticker1, ticker2, rvol1, rvol2):
    
    data = pd.DataFrame(zip(
        ticker1.date,
        ticker1.val,
        ticker2.val,
        ticker1.val - ticker2.val,
        rvol1 - rvol2
    ), columns = ['date', 'v1', 'v2', 'spread', 'rvspread'])
    
    r3 = data.spread.rolling(252*3, min_periods=1)
    r1 = data.spread.rolling(252, min_periods=1)

    data['Mean3Y'] = r3.mean()
    data['ZScore3Y'] = (data.spread - data.Mean3Y) / r3.std()
    data['Min3Y'] = r3.min()
    data['Max3Y'] = r3.max()
    data['Rank3Y'] = (data.spread - data.Min3Y) / (data.Max3Y - data.Min3Y)
    data['PctRank3Y'] = r3.apply(lambda x: pctrank(x, x.values[-1]) / 100)

    data['Mean'] = r1.mean()
    data['ZScore'] = (data.spread - data.Mean) / r1.std()
    data['Min'] = r1.min()
    data['Max'] = r1.max()
    data['Rank'] = (data.spread - data.Min) / (data.Max - data.Min)
    data['PctRank'] = r1.apply(lambda x: pctrank(x, x.values[-1]) / 100)

    data['Corr6M'] = data.v1.rolling(126, min_periods=1).corr(data.v2)
    data['Corr3M'] = data.v1.rolling(63, min_periods=1).corr(data.v2)
    
    data['Carry'] = data.rvspread - data.spread
    
    return data

In [24]:
rvx_vix = pair_stats(rvx, vix, rutrv, spxrv)
rvx_vix

,date,v1,v2,spread,rvspread,Mean3Y,ZScore3Y,Min3Y,Max3Y,Rank3Y,PctRank3Y,Mean,ZScore,Min,Max,Rank,PctRank,Corr6M,Corr3M,Carry
0,2010-01-04,26.55,20.04,6.51,NaN,6.510000,NaN,6.51,6.51,NaN,1.000000,6.510000,NaN,6.51,6.51,NaN,1.000000,NaN,NaN,NaN
1,2010-01-05,26.66,19.35,7.31,-0.205232,6.910000,0.707107,6.51,7.31,1.000000,1.000000,6.910000,0.707107,6.51,7.31,1.000000,1.000000,-1.000000,-1.000000,-7.515232
2,2010-01-06,25.64,19.16,6.48,-0.173751,6.766667,-0.608919,6.48,7.31,0.000000,0.333333,6.766667,-0.608919,6.48,7.31,0.000000,0.333333,0.590614,0.590614,-6.653751
3,2010-01-07,24.82,19.06,5.76,0.598973,6.515000,-1.192122,5.76,7.31,0.000000,0.250000,6.515000,-1.192122,5.76,7.31,0.000000,0.250000,0.706301,0.706301,-5.161027
4,2010-01-08,23.55,18.13,5.42,0.717748,6.296000,-1.191390,5.42,7.31,0.000000,0.200000,6.296000,-1.191390,5.42,7.31,0.000000,0.200000,0.905277,0.905277,-4.702252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,2020-11-17,29.54,22.71,6.83,7.160432,3.830873,0.965392,-6.02,16.26,0.576750,0.858466,6.051667,0.194031,-2.92,16.26,0.508342,0.579365,0.858336,0.940712,0.330432
2739,2020-11-18,30.68,23.84,6.84,7.169929,3.835198,0.966623,-6.02,16.26,0.577199,0.858466,6.065635,0.193206,-2.92,16.26,0.508863,0.579365,0.859624,0.939294,0.329929
2740,2020-11-19,30.17,23.11,7.06,7.130043,3.840079,1.035105,-6.02,16.26,0.587074,0.867725,6.080595,0.244570,-2.92,16.26,0.520334,0.603175,0.861661,0.937920,0.070043
2741,2020-11-20,29.50,23.70,5.80,6.523497,3.841587,0.629432,-6.02,16.26,0.530521,0.822751,6.090397,-0.072583,-2.92,16.26,0.454640,0.496032,0.863051,0.937202,0.723497


In [25]:
21.54 * (63 / 260) * (252 / 63)

20.877230769230767

In [26]:
spxrv3m

0             NaN
1        0.622167
2        0.631655
3        1.018244
4        1.169632
          ...    
2738    20.892132
2739    21.011776
2740    21.015310
2741    20.964411
2742    20.982242
Name: val, Length: 2743, dtype: float64